In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json

In [2]:
df = pd.read_excel(r"C:\Users\Minfy\Downloads\Datasets\Bank_Personal_Loan_Modelling.xlsx" , sheet_name="Data")
new_df = pd.read_csv(r"C:\Users\Minfy\Downloads\New Customer Bank_Personal_Loan.csv")

In [3]:
# df.drop(["Personal Loan"] , inplace=True , axis=1)

In [4]:
df.drop(['ID','ZIP Code','Personal Loan'] , inplace=True , axis=1)
new_df.drop(['ID','ZIP Code'] , inplace=True , axis=1)


In [5]:
from evidently import Report
from evidently.presets import DataDriftPreset

In [16]:
report = Report([DataDriftPreset()])

In [14]:

df_train , df_test = train_test_split(df)

In [17]:
df_train_test_eval = report.run(df_train,df_test) 
df_train_test_eval.save_html("df_train_test_eval.html")

In [7]:
report_new = Report([DataDriftPreset()])

In [22]:
df_hist_df_new = report_new.run(df , new_df)
df_hist_df_new.save_html("df_historical_vs_new_df.html")

<h2> Code for adding New data vs Historical Data into MLflow </h2>

In [12]:
import mlflow
import re

def sanitize_name(name):
    return re.sub(r"[^\w\-/ .]", "_", name)


report_data = json.loads(df_hist_df_new.json())

mlflow.set_experiment("evidently_historical_vs_newdata")
with mlflow.start_run():
    for i in report_data.get("metrics", []):
        metric_id = i.get("metric_id", "")
        value = i.get("value", None)

        if metric_id.startswith("Drifted"):
            mlflow.log_metric("Number_of_driftedcolumns", value['count'])
        else:
           
            clean_metric_id = sanitize_name(metric_id.lower().replace(" ", "_"))
            mlflow.log_metric(clean_metric_id, value)

        print(value)

2025/07/03 09:30:47 INFO mlflow.tracking.fluent: Experiment with name 'evidently_historical_vs_newdata' does not exist. Creating a new experiment.
2025/07/03 09:30:47 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Exa

{'count': 3.0, 'share': 0.2727272727272727}
0.02418149798406314
0.0754008440112274
0.45251348912712985
0.6104785358474184
0.8776517059542743
0.0
1.7600161054519227e-124
0.18613603663787415
0.32616739833182207
0.3050098088783806
0.05348648711780135


<h2> Code for adding Train vs Test data into MLflow </h2>

In [18]:
report_data = json.loads(df_train_test_eval.json())

mlflow.set_experiment("evidently_train_vs_test")
with mlflow.start_run():
    for i in report_data.get("metrics", []):
        metric_id = i.get("metric_id", "")
        value = i.get("value", None)

        if metric_id.startswith("Drifted"):
            mlflow.log_metric("Number_of_driftedcolumns", value['count'])
        else:
           
            clean_metric_id = sanitize_name(metric_id.lower().replace(" ", "_"))
            mlflow.log_metric(clean_metric_id, value)

        print(value)

2025/07/03 09:36:31 INFO mlflow.tracking.fluent: Experiment with name 'evidently_train_vs_test' does not exist. Creating a new experiment.


{'count': 0.0, 'share': 0.0}
0.020227020569669282
0.023285474959749086
0.024561557448814313
0.0653490515068191
0.031389802479988814
0.027762492322887137
0.001554393449570352
0.0006163137174247002
0.003978384969377368
0.016875768462873714
0.009548903155295411
